In [1]:
import numpy as np
import pandas as pd
import requests
from io import StringIO
from bs4 import BeautifulSoup as BS
from time import sleep
from tqdm import tqdm
import re

In [13]:
fda_df = pd.read_csv('Data/fdalabel-query-327575.csv')

In [14]:
fda_df.head(10)

,Labeling Type,Dosage Form(s),Route(s) of Administration,Marketing Category,Application Number(s),Trade Name,Generic/Proper Name(s),Most Recent SPL Date (YYYY/MM/DD),Established Pharmacologic Class(es),Initial U.S. Approval,...,Marketing Date(s) (YYYY/MM/DD),NDC(s),Active Ingredient UNII(s),Active Ingredient(s),Active Moiety Name(s),Active Moiety UNII(s),FDALabel Link,DailyMed Link,DailyMed PDF Link,SET ID
0,HUMAN PRESCRIPTION DRUG LABEL,SOLUTION,ORAL,NDA,212690,XYWAV,"(CALCIUM, MAGNESIUM, POTASSIUM, AND SODIUM OXYBATES)",20210823,Central Nervous System Depressant,2002.0,...,2020/11/02-,68727-150,7G33012534; 8W24SYD6ZI; G983HLV265; S8NKF3H3KT,Calcium Oxybate; Magnesium Oxybate; Potassium Oxybate; SODIUM OXYBATE,4-HYDROXYBUTANOIC ACID,30IW36W5B2,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/1e0ae43a-037f-42af-8e23-a0e51d75abe8/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=1e0ae43a-037f-42af-8e23-a0e51d75abe8,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=1e0ae43a-037f-42af-8e23-a0e51d75abe8,1e0ae43a-037f-42af-8e23-a0e51d75abe8
1,HUMAN PRESCRIPTION DRUG LABEL,"INJECTION, SOLUTION",INFILTRATION; PERINEURAL,NDA,009435,(CHLOROPROCAINE HCI,(CHLOROPROCAINE HCI,20200222,Ester Local Anesthetic,NaN,...,2019/10/13-,51662-1407,LT7Z1YW11H,CHLOROPROCAINE HYDROCHLORIDE,CHLOROPROCAINE; EDETIC ACID,5YVB0POT2H; 9G34HU7RV0,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/94d13f48-fbca-67b4-e053-2995a90a791b/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=94d13f48-fbca-67b4-e053-2995a90a791b,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=94d13f48-fbca-67b4-e053-2995a90a791b,94d13f48-fbca-67b4-e053-2995a90a791b
2,HUMAN PRESCRIPTION DRUG LABEL,SOLUTION,AURICULAR (OTIC),NDA,208251,OTOVEL,(CIPROFLOXACIN AND FLUOCINOLONE ACETONIDE),20190912,Corticosteroid; Quinolone Antimicrobial,2016.0,...,2016/04/29-,24338-080,0CD5FD6S2M; 5E8K9I0O4U,CIPROFLOXACIN; FLUOCINOLONE ACETONIDE,CIPROFLOXACIN; FLUOCINOLONE ACETONIDE,0CD5FD6S2M; 5E8K9I0O4U,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/1a7e4806-3138-4b7a-b798-a99e163ee846/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=1a7e4806-3138-4b7a-b798-a99e163ee846,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=1a7e4806-3138-4b7a-b798-a99e163ee846,1a7e4806-3138-4b7a-b798-a99e163ee846
3,HUMAN PRESCRIPTION DRUG LABEL,KIT,INTRAVENOUS,BLA,125611,REBINYN,"(COAGULATION FACTOR IX (RECOMBINANT), GLYCOPEGYLATED)",20200630,Human Blood Coagulation Factor,2017.0,...,2017/12/01-,0169-7901; 0169-7902; 0169-7905,382L14738L,COAGULATION FACTOR IX RECOMBINANT HUMAN,COAGULATION FACTOR IX RECOMBINANT HUMAN,382L14738L,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/0ea37235-35fd-410d-b8c4-40ba15fe1294/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=0ea37235-35fd-410d-b8c4-40ba15fe1294,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=0ea37235-35fd-410d-b8c4-40ba15fe1294,0ea37235-35fd-410d-b8c4-40ba15fe1294
4,HUMAN PRESCRIPTION DRUG LABEL,"INJECTION, POWDER, LYOPHILIZED, FOR SUSPENSION",INTRAVENOUS,NDA,209401,VYXEOS,(DAUNORUBICIN AND CYTARABINE) LIPOSOME,20210413,Anthracycline Topoisomerase Inhibitor; Nucleoside Metabolic Inhibitor,2017.0,...,2017/08/03-,68727-745,04079A1RDZ; ZS7284E0ZP,CYTARABINE; DAUNORUBICIN,CYTARABINE; DAUNORUBICIN,04079A1RDZ; ZS7284E0ZP,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/7ea701ce-e7d3-4349-a9c2-642a501d45c8/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=7ea701ce-e7d3-4349-a9c2-642a501d45c8,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=7ea701ce-e7d3-4349-a9c2-642a501d45c8,7ea701ce-e7d3-4349-a9c2-642a501d45c8
5,HUMAN PRESCRIPTION DRUG LABEL,INJECTION,INTRAVENOUS,BLA,761107,GAMIFANT,(EMAPALUMAB-LZSG),20200630,Interferon gamma Blocker,2018.0,...,2019/05/17-; 2020/06/26-,66658-501; 66658-505; 66658-510,3S252O2Z4X,emapalumab,emapalumab,3S252O2Z4X,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/a865e0ef-8685-4f69-8838-648c4f3ba

In [15]:
fda_df['Trade Name'] = fda_df['Trade Name'].str.lower()

In [16]:
fda_df = fda_df.sort_values('Most Recent SPL Date (YYYY/MM/DD)', ascending = False)

In [17]:
fda_df.drop_duplicates(subset='Trade Name')

,Labeling Type,Dosage Form(s),Route(s) of Administration,Marketing Category,Application Number(s),Trade Name,Generic/Proper Name(s),Most Recent SPL Date (YYYY/MM/DD),Established Pharmacologic Class(es),Initial U.S. Approval,...,Marketing Date(s) (YYYY/MM/DD),NDC(s),Active Ingredient UNII(s),Active Ingredient(s),Active Moiety Name(s),Active Moiety UNII(s),FDALabel Link,DailyMed Link,DailyMed PDF Link,SET ID
10855,HUMAN PRESCRIPTION DRUG LABEL,TABLET,ORAL,Export only,NaN,cyproheptadine hydrochloride,CYPROHEPTADINE HYDROCHLORIDE,2010110126,NaN,NaN,...,2010/06/15-,62033-0346,NJ82J0F8QC,Cyproheptadine Hydrochloride,Cyproheptadine,2YHB6175DO,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/f90faaa3-f40f-4653-8fb8-144d61c6b1d4/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=f90faaa3-f40f-4653-8fb8-144d61c6b1d4,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=f90faaa3-f40f-4653-8fb8-144d61c6b1d4,f90faaa3-f40f-4653-8fb8-144d61c6b1d4
17740,HUMAN PRESCRIPTION DRUG LABEL,"TABLET, FILM COATED",ORAL,ANDA,077525,gabapentin,GABAPENTIN,20220321,Anti-epileptic Agent,1993.0,...,2006/08/24-,71610-069,6CW7F3G59X,GABAPENTIN,GABAPENTIN,6CW7F3G59X,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/7790d906-af26-4e60-bc6f-e906ec03cc24/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=7790d906-af26-4e60-bc6f-e906ec03cc24,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=7790d906-af26-4e60-bc6f-e906ec03cc24,7790d906-af26-4e60-bc6f-e906ec03cc24
22866,HUMAN PRESCRIPTION DRUG LABEL,"TABLET, FILM COATED",ORAL,ANDA,078106,levetiracetam,LEVETIRACETAM,20220108,NaN,1999.0,...,2012/09/11-,43547-221; 43547-222; 43547-223; 43547-224,44YRR34555,LEVETIRACETAM,LEVETIRACETAM,44YRR34555,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/dd713e60-478c-43cd-b680-ff594a0b3864/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=dd713e60-478c-43cd-b680-ff594a0b3864,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=dd713e60-478c-43cd-b680-ff594a0b3864,dd713e60-478c-43cd-b680-ff594a0b3864
5728,HUMAN PRESCRIPTION DRUG LABEL,"TABLET, FILM COATED, EXTENDED RELEASE",ORAL,ANDA,077415,bupropion hydrochloride,BUPROPION HYDROCHLORIDE,20220101,Aminoketone,1985.0,...,2009/05/01-,63187-713,ZG7E5POY8O,BUPROPION HYDROCHLORIDE,BUPROPION,01ZG3TPX31,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/e57451fd-aba5-47d6-9892-ffe2a6021b6f/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=e57451fd-aba5-47d6-9892-ffe2a6021b6f,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=e57451fd-aba5-47d6-9892-ffe2a6021b6f,e57451fd-aba5-47d6-9892-ffe2a6021b6f
17439,HUMAN PRESCRIPTION DRUG LABEL,"INJECTION, POWDER, LYOPHILIZED, FOR SOLUTION",INTRAVENOUS,ANDA,204025,fosaprepitant,FOSAPREPITANT,20211231,Substance P/Neurokinin-1 Receptor Antagonist,2008.0,...,2020/10/22-,16729-240,D35FM8T64X,FOSAPREPITANT DIMEGLUMINE,APREPITANT,1NF15YR6UY,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/4a3bac1c-1702-4963-b967-f1699f03cc6f/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=4a3bac1c-1702-4963-b967-f1699f03cc6f,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=4a3bac1c-1702-4963-b967-f1699f03cc6f,4a3bac1c-1702-4963-b967-f1699f03cc6f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33647,HUMAN PRESCRIPTION DRUG LABEL,CAPSULE,ORAL,NaN,NaN,urobiotic,"OXYTETRACYCLINE HYDROCHLORIDE, SULFAMETHIZOLE AND PHENAZOPYRIDINE HYDROCHLORIDE",20051214,NaN,NaN,...,NaN,0049-0920,25W8454H16; 2IUY41693Z; 4U7K4N52ZM,oxytetracycline hydrochloride; phenazopyridine hydrochloride; sulfamethizole,oxytetracycline; phenazopyridine; sulfamethizole,25W8454H16; X20I9EN955,http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/be27854a-a805-4300-9729-accd1b7f226f/spl-doc,http://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=be27854a-a805-4300-9729-accd1b7f226f,http://dailymed.nlm.nih.gov/dailymed/downloadpdffile.cfm?setId=be27854a-a805-4300-9729-accd1b7f226f,be27854a

In [18]:
fda_df['webscraping_name'] = fda_df['Trade Name'].str.extract('(\w+)')

In [19]:
pd.set_option('display.max_colwidth', None)
fda_df['FDALabel Link']

10855    http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/f90faaa3-f40f-4653-8fb8-144d61c6b1d4/spl-doc
17740    http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/7790d906-af26-4e60-bc6f-e906ec03cc24/spl-doc
22866    http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/dd713e60-478c-43cd-b680-ff594a0b3864/spl-doc
5728     http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/e57451fd-aba5-47d6-9892-ffe2a6021b6f/spl-doc
17439    http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/4a3bac1c-1702-4963-b967-f1699f03cc6f/spl-doc
                                                        ...                                                
26675    http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/40e35fa9-9cfe-04f1-e054-00144ff8d46c/spl-doc
40691    http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/1cd07b54-f66f-4040-9058-462324267196/spl-doc
22289    http://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/40d145e6-13f7-41ac-e054-00144ff8d46c/spl-doc
36580    http://nctr-crs.fda

In [20]:
drugz = 'teflaro'

In [21]:
endpoint = f'https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/{drugz}-side-effects.html'

In [22]:
response = requests.get(endpoint)

In [23]:
response

<Response [200]>

In [24]:
soup = BS(response.text)

In [25]:
soup.find_all(['h3','i','li'])

[<i class="icon icon-cross"></i>,
 <li>
 <a href="https://web.archive.org/web/20160608042942/https://www.facebook.com/Drugscom" target="_blank"><i class="icon icon-facebook"></i></a>
 <a href="https://web.archive.org/web/20160608042942/https://twitter.com/drugscom" target="_blank"><i class="icon icon-twitter"></i></a>
 <a href="https://web.archive.org/web/20160608042942/https://plus.google.com/+drugscom?rel=publisher" target="_blank"><i class="icon icon-googleplus"></i></a>
 </li>,
 <i class="icon icon-facebook"></i>,
 <i class="icon icon-twitter"></i>,
 <i class="icon icon-googleplus"></i>,
 <li class="nav_item dropnav menu_account logged-out">
 <a class="nav_link" href="https://web.archive.org/web/20160608042942/https://www.drugs.com/account/login/" rel="nofollow"><i class="icon icon-user"></i>Sign In</a>
 <div class="nav_folder">
 <ul class="sub-menu">
 <li><a class="input-button btn-block" href="https://web.archive.org/web/20160608042942/https://www.drugs.com/account/login/" rel="n

In [26]:
#COME BACK TO THIS TO SEE IF ITS POSSIBLE. THE WAY TO DO IT WOULD BE GRAB FIRST 2 H3, THEN GRAB THE Is and put them into LIST,
#THEN GRAB LIs UNDER EACH I AND PUT THEM INTO A LIST, THEN ZIP LISTS 
# list1 = []
# for span in soup.find_all(['h3','i', 'li'], attrs={'class':None},string = True):
#      text = span.next_element

#      if text:
#          list1.append(span)

        
# x = list1.index('More common')
# y = list1.index('General')
# list1[x:y]
        
        

In [27]:
list1 = soup.find_all(['h3','i', 'li'], attrs={'class':None},string = True)

In [28]:
str_list1 = [str(i) for i in list1]

In [29]:
#index1 = str_list1.index('<h3>Side effects requiring immediate medical attention</h3>')

In [30]:
#index2 = str_list1.index([x for x in str_list1 if f'<i>Applies to {drugz}' in x][0])

In [31]:
#q = list1[index1:index2][0]

In [32]:
#q.name


In [34]:
h3 = ''
i = ''
li = ''
SE_df = pd.DataFrame()
results = []
garbagelist = []
for drugz in  list(fda_df['webscraping_name'].unique()):
    URL123 = f'https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/{drugz}-side-effects.html'
    res1 = requests.get(URL123)
    soup = BS(res1.text)
    list1 = soup.find_all(['h3','i', 'li'], attrs={'class':None},string = True)
    print(URL123)
    str_list1 = [str(i) for i in list1]
    if ('<h3>Side effects requiring immediate medical attention</h3>' in str_list1) & ('<h3>General</h3>' in str_list1):
        index_first = str_list1.index('<h3>Side effects requiring immediate medical attention</h3>')
        #index_last = str_list1.index([x for x in str_list1 if f'<i>Applies to {drugz}' in x][0])
        index_last = str_list1.index('<h3>General</h3>')
        for ele in list1[index_first:index_last]:
            if ele.name == 'h3':
                h3 = ele.text
            elif ele.name == 'i':
                i = ele.text
            else:
                li = ele.text
                results.append((drugz,h3,i,li))
    elif ('<h3>Side effects requiring immediate medical attention</h3>' in str_list1) :
        if ele.name == 'h3':
                h3 = ele.text
        elif ele.name == 'i':
                i = ele.text
        else:
                li = ele.text
                results.append((drugz,h3,i,li))
    else:
        results.append((drugz,'NA','NA','NA'))

https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/cyproheptadine-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/gabapentin-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/levetiracetam-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/bupropion-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/fosaprepitant-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/paclitaxel-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/teflaro-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/dimethyl-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/aricept-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/pravastatin-side-effects.html
https://web.archiv

https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/amlodipine-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/nystatin-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/fluoxetine-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/meloxicam-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/tizanidine-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/amoxicillin-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/ribavirin-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/kaletra-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/enalapril-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/clodan-side-effects.html
https://web.archive.org/web/202

https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/adapalene-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/vytorin-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/tri-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/triamterene-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/potassium-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/carvedilol-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/risperdal-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/nifedipine-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/diovan-side-effects.html
https://web.archive.org/web/20211121022625/https://www.drugs.com/sfx/lorazepam-side-effects.html
https://web.archive.org/web/202111210

ConnectionError: HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded with url: /web/20211121022625/https://www.drugs.com/sfx/medroxyprogesterone-side-effects.html (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C078178370>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [42]:
h3 = ''
i = ''
li = ''
SE_df = pd.DataFrame()
results = []
garbagelist = []
for drugz in  list(fda_df['webscraping_name'].unique()):
    URL123 = f'https://www.drugs.com/sfx/{drugz}-side-effects.html'
    res1 = requests.get(URL123)
    soup = BS(res1.text)
    list1 = soup.find_all(['h3','i', 'li'], attrs={'class':None},string = True)
    print(URL123)
    str_list1 = [str(i) for i in list1]
    if ('<h3>Side effects requiring immediate medical attention</h3>' in str_list1) & ('<h3>General</h3>' in str_list1):
        index_first = str_list1.index('<h3>Side effects requiring immediate medical attention</h3>')
        #index_last = str_list1.index([x for x in str_list1 if f'<i>Applies to {drugz}' in x][0])
        index_last = str_list1.index('<h3>General</h3>')
        for ele in list1[index_first:index_last]:
            if ele.name == 'h3':
                h3 = ele.text
            elif ele.name == 'i':
                i = ele.text
            else:
                li = ele.text
                results.append((drugz,h3,i,li))
    elif ('<h3>Side effects requiring immediate medical attention</h3>' in str_list1) :
        if ele.name == 'h3':
                h3 = ele.text
        elif ele.name == 'i':
                i = ele.text
        else:
                li = ele.text
                results.append((drugz,h3,i,li))
    else:
        results.append((drugz,'NA','NA','NA'))

https://www.drugs.com/sfx/cyproheptadine-side-effects.html
https://www.drugs.com/sfx/gabapentin-side-effects.html
https://www.drugs.com/sfx/levetiracetam-side-effects.html
https://www.drugs.com/sfx/bupropion-side-effects.html
https://www.drugs.com/sfx/fosaprepitant-side-effects.html
https://www.drugs.com/sfx/paclitaxel-side-effects.html
https://www.drugs.com/sfx/teflaro-side-effects.html
https://www.drugs.com/sfx/dimethyl-side-effects.html
https://www.drugs.com/sfx/aricept-side-effects.html
https://www.drugs.com/sfx/pravastatin-side-effects.html
https://www.drugs.com/sfx/atorvastatin-side-effects.html
https://www.drugs.com/sfx/xenical-side-effects.html
https://www.drugs.com/sfx/licart-side-effects.html
https://www.drugs.com/sfx/valacyclovir-side-effects.html
https://www.drugs.com/sfx/phentermine-side-effects.html
https://www.drugs.com/sfx/trandolapril-side-effects.html
https://www.drugs.com/sfx/azithromycin-side-effects.html
https://www.drugs.com/sfx/celecoxib-side-effects.html
https:/

https://www.drugs.com/sfx/belladonna-side-effects.html
https://www.drugs.com/sfx/cyanocobalamin-side-effects.html
https://www.drugs.com/sfx/celebrex-side-effects.html
https://www.drugs.com/sfx/malarone-side-effects.html
https://www.drugs.com/sfx/diphenoxylate-side-effects.html
https://www.drugs.com/sfx/lidocaine-side-effects.html
https://www.drugs.com/sfx/letrozole-side-effects.html
https://www.drugs.com/sfx/crestor-side-effects.html
https://www.drugs.com/sfx/sulindac-side-effects.html
https://www.drugs.com/sfx/granisetron-side-effects.html
https://www.drugs.com/sfx/clindamycin-side-effects.html
https://www.drugs.com/sfx/solifenacin-side-effects.html
https://www.drugs.com/sfx/lisinopril-side-effects.html
https://www.drugs.com/sfx/cefdinir-side-effects.html
https://www.drugs.com/sfx/sumatriptan-side-effects.html
https://www.drugs.com/sfx/fluconazole-side-effects.html
https://www.drugs.com/sfx/viagra-side-effects.html
https://www.drugs.com/sfx/alprazolam-side-effects.html
https://www.dru

https://www.drugs.com/sfx/diflucan-side-effects.html
https://www.drugs.com/sfx/megestrol-side-effects.html
https://www.drugs.com/sfx/levothyroxine-side-effects.html
https://www.drugs.com/sfx/acarbose-side-effects.html
https://www.drugs.com/sfx/arava-side-effects.html
https://www.drugs.com/sfx/hemabate-side-effects.html
https://www.drugs.com/sfx/cpda-side-effects.html
https://www.drugs.com/sfx/trileptal-side-effects.html
https://www.drugs.com/sfx/digoxin-side-effects.html
https://www.drugs.com/sfx/sotalol-side-effects.html
https://www.drugs.com/sfx/lysteda-side-effects.html
https://www.drugs.com/sfx/chlorpromazine-side-effects.html
https://www.drugs.com/sfx/detrol-side-effects.html
https://www.drugs.com/sfx/chlorthalidone-side-effects.html
https://www.drugs.com/sfx/misoprostol-side-effects.html
https://www.drugs.com/sfx/dalfampridine-side-effects.html
https://www.drugs.com/sfx/amidate-side-effects.html
https://www.drugs.com/sfx/erythrocin-side-effects.html
https://www.drugs.com/sfx/kepp

https://www.drugs.com/sfx/oseltamivir-side-effects.html
https://www.drugs.com/sfx/fluphenazine-side-effects.html
https://www.drugs.com/sfx/nitrous-side-effects.html
https://www.drugs.com/sfx/dronabinol-side-effects.html
https://www.drugs.com/sfx/desvenlafaxine-side-effects.html
https://www.drugs.com/sfx/methylergonovine-side-effects.html
https://www.drugs.com/sfx/flecainide-side-effects.html
https://www.drugs.com/sfx/nivestym-side-effects.html
https://www.drugs.com/sfx/methylphenidate-side-effects.html
https://www.drugs.com/sfx/valganciclovir-side-effects.html
https://www.drugs.com/sfx/inrebic-side-effects.html
https://www.drugs.com/sfx/oxcarbazepine-side-effects.html
https://www.drugs.com/sfx/zilxi-side-effects.html
https://www.drugs.com/sfx/infant-side-effects.html
https://www.drugs.com/sfx/mometasone-side-effects.html
https://www.drugs.com/sfx/ampyra-side-effects.html
https://www.drugs.com/sfx/clozapine-side-effects.html
https://www.drugs.com/sfx/amzeeq-side-effects.html
https://www

https://www.drugs.com/sfx/halobetasol-side-effects.html
https://www.drugs.com/sfx/fluorouracil-side-effects.html
https://www.drugs.com/sfx/ipratropium-side-effects.html
https://www.drugs.com/sfx/avapro-side-effects.html
https://www.drugs.com/sfx/ethosuximide-side-effects.html
https://www.drugs.com/sfx/cialis-side-effects.html
https://www.drugs.com/sfx/intersol-side-effects.html
https://www.drugs.com/sfx/semglee-side-effects.html
https://www.drugs.com/sfx/trihexyphenidyl-side-effects.html
https://www.drugs.com/sfx/avalide-side-effects.html
https://www.drugs.com/sfx/cablivi-side-effects.html
https://www.drugs.com/sfx/rydapt-side-effects.html
https://www.drugs.com/sfx/zolinza-side-effects.html
https://www.drugs.com/sfx/janumet-side-effects.html
https://www.drugs.com/sfx/zepatier-side-effects.html
https://www.drugs.com/sfx/pioglitazone-side-effects.html
https://www.drugs.com/sfx/gemcitabine-side-effects.html
https://www.drugs.com/sfx/pancreaze-side-effects.html
https://www.drugs.com/sfx/ph

...

https://www.drugs.com/sfx/ethynodiol-side-effects.html
https://www.drugs.com/sfx/portrazza-side-effects.html
https://www.drugs.com/sfx/vanos-side-effects.html
https://www.drugs.com/sfx/kariva-side-effects.html
https://www.drugs.com/sfx/toposar-side-effects.html
https://www.drugs.com/sfx/mircette-side-effects.html
https://www.drugs.com/sfx/lessina-side-effects.html
https://www.drugs.com/sfx/portia-side-effects.html
https://www.drugs.com/sfx/monodox-side-effects.html
https://www.drugs.com/sfx/ocuflox-side-effects.html
https://www.drugs.com/sfx/primacare-side-effects.html
https://www.drugs.com/sfx/relegard-side-effects.html
https://www.drugs.com/sfx/cortane-side-effects.html
https://www.drugs.com/sfx/anodyne-side-effects.html
https://www.drugs.com/sfx/jolivette-side-effects.html
https://www.drugs.com/sfx/paingo-side-effects.html
https://www.drugs.com/sfx/ziana-side-effects.html
https://www.drugs.com/sfx/saphris-side-effects.html
https://www.drugs.com/sfx/benzaclin-side-effects.html
https:

https://www.drugs.com/sfx/immune-side-effects.html
https://www.drugs.com/sfx/lidopin-side-effects.html
https://www.drugs.com/sfx/loratadine-side-effects.html
https://www.drugs.com/sfx/ciferex-side-effects.html
https://www.drugs.com/sfx/zithranol-side-effects.html
https://www.drugs.com/sfx/melamin-side-effects.html
https://www.drugs.com/sfx/melamix-side-effects.html
https://www.drugs.com/sfx/hycet-side-effects.html
https://www.drugs.com/sfx/falessa-side-effects.html
https://www.drugs.com/sfx/active-side-effects.html
https://www.drugs.com/sfx/momlove-side-effects.html
https://www.drugs.com/sfx/esmenopa-side-effects.html
https://www.drugs.com/sfx/amoxicillan-side-effects.html
https://www.drugs.com/sfx/bleph-side-effects.html
https://www.drugs.com/sfx/metaproterenol-side-effects.html
https://www.drugs.com/sfx/elastiderm-side-effects.html
https://www.drugs.com/sfx/rectacort-side-effects.html
https://www.drugs.com/sfx/bioferr-side-effects.html
https://www.drugs.com/sfx/premasol-side-effects.

https://www.drugs.com/sfx/tarka-side-effects.html
https://www.drugs.com/sfx/rhinocort-side-effects.html
https://www.drugs.com/sfx/lymphazurin-side-effects.html
https://www.drugs.com/sfx/prometh-side-effects.html
https://www.drugs.com/sfx/monobasic-side-effects.html
https://www.drugs.com/sfx/opana-side-effects.html
https://www.drugs.com/sfx/sectral-side-effects.html
https://www.drugs.com/sfx/fluress-side-effects.html
https://www.drugs.com/sfx/teveten-side-effects.html
https://www.drugs.com/sfx/meijer-side-effects.html
https://www.drugs.com/sfx/requip-side-effects.html
https://www.drugs.com/sfx/octocaine-side-effects.html
https://www.drugs.com/sfx/avelox-side-effects.html
https://www.drugs.com/sfx/cnl8-side-effects.html
https://www.drugs.com/sfx/reaphirm-side-effects.html
https://www.drugs.com/sfx/one-side-effects.html
https://www.drugs.com/sfx/gap-side-effects.html
https://www.drugs.com/sfx/felbatol-side-effects.html
https://www.drugs.com/sfx/proairhfa-side-effects.html
https://www.drug

https://www.drugs.com/sfx/zerit-side-effects.html
https://www.drugs.com/sfx/simcor-side-effects.html
https://www.drugs.com/sfx/atroventhfa-side-effects.html
https://www.drugs.com/sfx/zydone-side-effects.html
https://www.drugs.com/sfx/alba-side-effects.html
https://www.drugs.com/sfx/proquin-side-effects.html
https://www.drugs.com/sfx/bioelements-side-effects.html
https://www.drugs.com/sfx/nifediac-side-effects.html
https://www.drugs.com/sfx/paser-side-effects.html
https://www.drugs.com/sfx/gris-side-effects.html
https://www.drugs.com/sfx/sonata-side-effects.html
https://www.drugs.com/sfx/proamatine-side-effects.html
https://www.drugs.com/sfx/recombinate-side-effects.html
https://www.drugs.com/sfx/tramapap-side-effects.html
https://www.drugs.com/sfx/la-side-effects.html
https://www.drugs.com/sfx/astelin-side-effects.html
https://www.drugs.com/sfx/xyzal-side-effects.html
https://www.drugs.com/sfx/mebendazole-side-effects.html
https://www.drugs.com/sfx/orbivan-side-effects.html
https://www

https://www.drugs.com/sfx/statrol-side-effects.html
https://www.drugs.com/sfx/maxaquin-side-effects.html
https://www.drugs.com/sfx/endrate-side-effects.html
https://www.drugs.com/sfx/neutrexin-side-effects.html
https://www.drugs.com/sfx/fluor-side-effects.html
https://www.drugs.com/sfx/viadur-side-effects.html
https://www.drugs.com/sfx/tolinase-side-effects.html
https://www.drugs.com/sfx/terra-side-effects.html
https://www.drugs.com/sfx/ismotic-side-effects.html
https://www.drugs.com/sfx/hms-side-effects.html
https://www.drugs.com/sfx/amphotec-side-effects.html
https://www.drugs.com/sfx/profenal-side-effects.html
https://www.drugs.com/sfx/trovan-side-effects.html
https://www.drugs.com/sfx/florone-side-effects.html
https://www.drugs.com/sfx/halotestin-side-effects.html
https://www.drugs.com/sfx/renese-side-effects.html
https://www.drugs.com/sfx/mykrox-side-effects.html
https://www.drugs.com/sfx/livostin-side-effects.html
https://www.drugs.com/sfx/kemstro-side-effects.html
https://www.dr

In [43]:
SE_df = pd.DataFrame(results)

In [44]:
SE_df = SE_df.rename(columns={0:'webscraping_name', 2:'medical_attention',2:'commonality', 3:'Side_effect'})

In [45]:
SE_df = SE_df.rename(columns = {1:'medical_attention'})

In [47]:
SE_df

,webscraping_name,medical_attention,commonality,Side_effect
0,cyproheptadine,NA,NA,NA
1,gabapentin,Side effects requiring immediate medical attention,More common,Clumsiness or unsteadiness
2,gabapentin,Side effects requiring immediate medical attention,More common,"continuous, uncontrolled, back-and-forth, or rolling eye movements"
3,gabapentin,Side effects requiring immediate medical attention,More common in children,anxiety
4,gabapentin,Side effects requiring immediate medical attention,More common in children,concentration problems and change in school performance
...,...,...,...,...
119558,minitran,Side effects not requiring immediate medical attention,Less common,stuffy nose
119559,minitran,Side effects not requiring immediate medical attention,Less common,"tender, swollen glands in the neck"
119560,minitran,Side effects not requiring immediate medical attention,Less common,voice changes
119561,urobiotic,NA,NA,NA


In [50]:
SE_df[SE_df['medical_attention'] != 'NA']

,webscraping_name,medical_attention,commonality,Side_effect
1,gabapentin,Side effects requiring immediate medical attention,More common,Clumsiness or unsteadiness
2,gabapentin,Side effects requiring immediate medical attention,More common,"continuous, uncontrolled, back-and-forth, or rolling eye movements"
3,gabapentin,Side effects requiring immediate medical attention,More common in children,anxiety
4,gabapentin,Side effects requiring immediate medical attention,More common in children,concentration problems and change in school performance
5,gabapentin,Side effects requiring immediate medical attention,More common in children,crying
...,...,...,...,...
119556,minitran,Side effects not requiring immediate medical attention,Less common,runny nose
119557,minitran,Side effects not requiring immediate medical attention,Less common,sneezing
119558,minitran,Side effects not requiring immediate medical attention,Less common,stuffy nose
119559,minitran,Side effects not requiring immediate medical attention,Less common,"tender, swollen glands in the neck"


In [48]:
SE_df.to_csv('Data/SE_df_update.csv')

In [ ]:
side_efx = soup.find('p', attrs = {'class':False})

In [ ]:
side_efx = side_efx.text

In [ ]:
side_efx = side_efx.split('.')

In [ ]:
side_efx[0:2]
common_se = side_efx[0]
other_se = side_efx[1]

In [ ]:
common_se = common_se.split(':')
other_se = other_se.split(':')

In [ ]:
def Convert(lst):
    res_dct = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)}
    return res_dct

In [ ]:
common_se_dict = Convert(common_se)
other_se_dict = Convert(other_se)

In [ ]:
common_se_dict

In [ ]:
fda_df.head()

In [ ]:
# drug_name = {}
# sfx = {'webscraping_name':[],'Side Effects':[]}
# common_sfx = []
# other_sfx = []
# drug_side_efx = pd.DataFrame()
# for drug in list(fda_df['webscraping_name'][0:10]):
#     URL = f'https://www.drugs.com/sfx/{drug}-side-effects.html'
#     res = requests.get(URL)
#     soup_obj = BS(res.text)
#     drug_name = {'webscraping_name':f'{drug}'}
#     side_effect = soup_obj.find('p', attrs = {'class':False})
#     side_eff = side_effect.text
#     #side_efx = side_eff.split('.')
# #     common_sfx = side_efx[0]
# #     other_sfx = side_efx[1]
# #     common_sfx= common_sfx.split(':')
# #     other_sfx = other_sfx.split(':')
# #     common_sfx_dict = Convert(common_sfx)
# #     other_sfx_dict = Convert(other_sfx)
# #     common_sfx_dict.update(other_sfx_dict)
# #     common_sfx_dict.update(drug_name)
#     sfx['webscraping_name'].append(f'{drug}')
#     sfx['Side Effects'].append(side_eff)
     
    
    

In [ ]:
# sfx_df = pd.DataFrame(sfx)

In [ ]:
# sfx_df

In [ ]:
# sfx_df[sfx_df['Side Effects'].str.contains('Applies')]

In [ ]:
# sfx_df

In [ ]:
# URL = 'https://www.drugs.com/sfx/cyproheptadine-side-effects.html'
# res = requests.get(URL)
# soup_obj = BS(res.text)

In [ ]:
# soup_obj.find_all('p', text = True)

In [ ]:
# common_sfx_dict

In [ ]:
# adverse_rxn.head()

In [2]:
 food_df = pd.read_csv('Data/food.csv')

In [3]:
food_nutrient_df = pd.read_csv('Data/food_nutrient.csv')

C:\Users\jacob\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
nutrient_df = pd.read_csv('Data/nutrient.csv').rename(columns = {'id':'nutrient_id'})

In [5]:
food_and_nutrient = pd.merge(food_nutrient_df, nutrient_df, on = 'nutrient_id')

In [6]:
food_and_nutrient.head()

,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,loq,footnote,min_year_acquired,name,unit_name,nutrient_nbr,rank
0,13338591,323505,2048,35.0,NaN,49.0,NaN,NaN,NaN,NaN,NaN,NaN,Energy (Atwater Specific Factors),KCAL,958.0,290.0
1,13338389,326196,2048,36.0,NaN,49.0,NaN,NaN,NaN,NaN,NaN,NaN,Energy (Atwater Specific Factors),KCAL,958.0,290.0
2,13338390,747447,2048,32.0,NaN,49.0,NaN,NaN,NaN,NaN,NaN,NaN,Energy (Atwater Specific Factors),KCAL,958.0,290.0
3,13338391,321611,2048,20.0,NaN,49.0,NaN,NaN,NaN,NaN,NaN,NaN,Energy (Atwater Specific Factors),KCAL,958.0,290.0
4,13338392,746769,2048,17.0,NaN,49.0,NaN,NaN,NaN,NaN,NaN,NaN,Energy (Atwater Specific Factors),KCAL,958.0,290.0


In [7]:
food = pd.merge(food_df,food_and_nutrient, on = 'fdc_id')

In [8]:
food.head()

,fdc_id,data_type,description,food_category_id,publication_date,id,nutrient_id,amount,data_points,derivation_id,min,max,median,loq,footnote,min_year_acquired,name,unit_name,nutrient_nbr,rank
0,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,2019-04-01,1283674,1003,5.88,1.0,46.0,NaN,NaN,NaN,NaN,NaN,NaN,Protein,G,203.0,600.0
1,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,2019-04-01,1283675,1007,3.50,1.0,46.0,NaN,NaN,NaN,NaN,NaN,NaN,Ash,G,207.0,1000.0
2,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,2019-04-01,1283676,1062,1286.00,0.0,49.0,NaN,NaN,NaN,NaN,NaN,NaN,Energy,kJ,268.0,400.0
3,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,2019-04-01,1283677,1079,1.20,1.0,46.0,NaN,NaN,NaN,NaN,NaN,NaN,"Fiber, total dietary",G,291.0,1200.0
4,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,2019-04-01,1283678,1089,2.12,1.0,46.0,NaN,NaN,NaN,NaN,NaN,NaN,"Iron, Fe",MG,303.0,5400.0


In [8]:
len(food['description'].unique())

284294

In [9]:
food = food.drop(columns = ['food_category_id','min','max','median','loq', 'footnote', 'min_year_acquired'])

In [49]:
food[food['name'].str.contains('Zinc')]

,fdc_id,data_type,description,publication_date,id,nutrient_id,amount,data_points,derivation_id,name,unit_name,nutrient_nbr,rank
157,167516,sr_legacy_food,"Waffles, buttermilk, frozen, ready-to-heat",2019-04-01,1283829,1095,0.480,2.0,1.0,"Zinc, Zn",MG,309.0,5900.0
268,167517,sr_legacy_food,"Waffle, buttermilk, frozen, ready-to-heat, toa...",2019-04-01,1283890,1095,0.480,6.0,1.0,"Zinc, Zn",MG,309.0,5900.0
352,167518,sr_legacy_food,"Waffle, buttermilk, frozen, ready-to-heat, mic...",2019-04-01,1284010,1095,0.580,2.0,1.0,"Zinc, Zn",MG,309.0,5900.0
424,167519,sr_legacy_food,"Waffle, plain, frozen, ready-to-heat, microwave",2019-04-01,1284062,1095,0.450,2.0,1.0,"Zinc, Zn",MG,309.0,5900.0
505,167520,sr_legacy_food,"Pie Crust, Cookie-type, Graham Cracker, Ready ...",2019-04-01,1284156,1095,1.250,3.0,1.0,"Zinc, Zn",MG,309.0,5900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7533313,2007132,sub_sample_food,"mushrooms, enoki",2021-10-15,24473366,1095,0.480,1.0,1.0,"Zinc, Zn",MG,309.0,5900.0
7533331,2007150,sub_sample_food,"mushrooms, enoki",2021-10-15,24473384,1095,0.462,1.0,1.0,"Zinc, Zn",MG,309.0,5900.0
7533349,2007168,sub_sample_food,"mushrooms, enoki",2021-10-15,24473402,1095,0.627,1.0,1.0,"Zinc, Zn",MG,309.0,5900.0
7533367,2007186,sub_sample_food,"mushrooms, enoki",2021-10-15,24473420,1095,0.444,1.0,1.0,"Zinc, Zn",MG,309.0,5900.0


In [ ]:
#se_df = pd.read_table('Data/meddra_all_label_se.tsv')

In [ ]:
#drug_code = pd.read_table('Data/drug_atc.tsv')

In [ ]:
#drug_names = pd.read_table('Data/drug_names.tsv')

In [ ]:
#drug_names.head(10)

In [ ]:
# fda_df[fda_df['Trade Name']=='CARNITINE']

In [ ]:
# THISURL = 'https://www.drugs.com/alpha/condition/a'

In [ ]:
# res1 = requests.get(THISURL)

In [ ]:
# soup_1 = BS(res1.text)

In [ ]:
# conditions = soup_1.select('.ddc-list-column-2 a')

In [ ]:
# conditions_list = list(conditions)

In [ ]:
# re.search(r'(>\w+\s?\w+?<)', str(conditions_list[1]))

In [ ]:
# conditions_list

In [ ]:
# x = len(conditions_list)
# cond_list = []
# for i in range(0,x):
#     cond_list.append(re.search(r'(>\w+\W?\s?\w+?\s?\w+?\s?\W?\w+?\W?<)', str(conditions_list[i])).group(1))
#     condition_list.append(cond_list[i].replace('>','')
#     cond_list[i].replace('<','')

In [ ]:
# cond_list

In [136]:
food_vitamin = food[(food['name'].str.contains('Vitamin'))|(food['name'].str.contains('Zinc'))]

In [137]:
food_vitamin = food_vitamin[(food_vitamin['unit_name'] != 'IU') ]
                           # & (food_vitamin['unit_name'] != 'UG')]
# .name.unique()
#.sort_values('amount', ascending = False)

In [138]:
food_vitamin['amount_per_100g'] = food_vitamin['amount']/1000

In [139]:
food_vitamin.head()

,fdc_id,data_type,description,publication_date,id,nutrient_id,amount,data_points,derivation_id,name,unit_name,nutrient_nbr,rank,amount_per_100g
29,167513,sr_legacy_food,"Pillsbury, Cinnamon Rolls with Icing, refriger...",2019-04-01,1283695,1106,0.0,0.0,51.0,"Vitamin A, RAE",UG,320.0,7420.0,0.0000
30,167513,sr_legacy_food,"Pillsbury, Cinnamon Rolls with Icing, refriger...",2019-04-01,1283696,1162,0.1,1.0,47.0,"Vitamin C, total ascorbic acid",MG,401.0,6300.0,0.0001
52,167515,sr_legacy_food,"George Weston Bakeries, Thomas English Muffins",2019-04-01,1283721,1106,0.0,0.0,NaN,"Vitamin A, RAE",UG,320.0,7420.0,0.0000
76,167516,sr_legacy_food,"Waffles, buttermilk, frozen, ready-to-heat",2019-04-01,1283738,1106,401.0,0.0,49.0,"Vitamin A, RAE",UG,320.0,7420.0,0.4010
77,167516,sr_legacy_food,"Waffles, buttermilk, frozen, ready-to-heat",2019-04-01,1283758,1162,0.0,2.0,1.0,"Vitamin C, total ascorbic acid",MG,401.0,6300.0,0.0000


In [120]:
food_vitamin.sort_values('amount_per_100g', ascending= False).name.unique()

array(['Vitamin B-12', 'Vitamin C, total ascorbic acid', 'Vitamin A, RAE',
       'Vitamin K (phylloquinone)', 'Vitamin E', 'Zinc, Zn',
       'Vitamin D3 (cholecalciferol)', 'Vitamin D (D2 + D3)',
       'Vitamin E (alpha-tocopherol)', 'Vitamin K (Dihydrophylloquinone)',
       'Vitamin B-6', 'Vitamin E, added', 'Vitamin K (Menaquinone-4)',
       'Vitamin D2 (ergocalciferol)', 'Vitamin B-12, added', 'Vitamin D4'],
      dtype=object)

In [140]:
food_vitamin = food_vitamin[(food_vitamin['name'] == 'Vitamin C, total ascorbic acid')|(food_vitamin['name'] == 'Vitamin A, RAE') |
(food_vitamin['name'] == 'Vitamin E (alpha-tocopherol)') |(food_vitamin['name'] == 'Vitamin E') |(food_vitamin['name'] == 'Zinc, Zn')]

In [127]:
food_vitamin.head()

,fdc_id,data_type,description,publication_date,id,nutrient_id,amount,data_points,derivation_id,name,unit_name,nutrient_nbr,rank,amount_per_100g
29,167513,sr_legacy_food,"Pillsbury, Cinnamon Rolls with Icing, refriger...",2019-04-01,1283695,1106,0.0,0.0,51.0,"Vitamin A, RAE",UG,320.0,7420.0,0.0000
30,167513,sr_legacy_food,"Pillsbury, Cinnamon Rolls with Icing, refriger...",2019-04-01,1283696,1162,0.1,1.0,47.0,"Vitamin C, total ascorbic acid",MG,401.0,6300.0,0.0001
52,167515,sr_legacy_food,"George Weston Bakeries, Thomas English Muffins",2019-04-01,1283721,1106,0.0,0.0,NaN,"Vitamin A, RAE",UG,320.0,7420.0,0.0000
76,167516,sr_legacy_food,"Waffles, buttermilk, frozen, ready-to-heat",2019-04-01,1283738,1106,401.0,0.0,49.0,"Vitamin A, RAE",UG,320.0,7420.0,0.4010
77,167516,sr_legacy_food,"Waffles, buttermilk, frozen, ready-to-heat",2019-04-01,1283758,1162,0.0,2.0,1.0,"Vitamin C, total ascorbic acid",MG,401.0,6300.0,0.0000


In [141]:
food_vitamin['amount_per_100g'] = np.where(food_vitamin['name'] != 'Vitamin A, RAE', food_vitamin['amount_per_100g'], food_vitamin['amount_per_100g']/1000)

In [146]:
food_vitamin = food_vitamin.drop(columns = ['fdc_id',
                             'data_type',
                             'publication_date',
                             'id',
                             'nutrient_id',
                             'data_points',
                             'derivation_id',
                             'nutrient_nbr', 
                            'rank'])

In [159]:
food_vitamin

,description,amount,name,unit_name,amount_per_100g
29,"Pillsbury, Cinnamon Rolls with Icing, refriger...",0.000,"Vitamin A, RAE",UG,0.000000
30,"Pillsbury, Cinnamon Rolls with Icing, refriger...",0.100,"Vitamin C, total ascorbic acid",MG,0.000100
52,"George Weston Bakeries, Thomas English Muffins",0.000,"Vitamin A, RAE",UG,0.000000
76,"Waffles, buttermilk, frozen, ready-to-heat",401.000,"Vitamin A, RAE",UG,0.000401
77,"Waffles, buttermilk, frozen, ready-to-heat",0.000,"Vitamin C, total ascorbic acid",MG,0.000000
...,...,...,...,...,...
7533313,"mushrooms, enoki",0.480,"Zinc, Zn",MG,0.000480
7533331,"mushrooms, enoki",0.462,"Zinc, Zn",MG,0.000462
7533349,"mushrooms, enoki",0.627,"Zinc, Zn",MG,0.000627
7533367,"mushrooms, enoki",0.444,"Zinc, Zn",MG,0.000444


In [161]:
food_vit_pivot = food_vitamin.pivot_table(index = 'description',columns = 'name', values = 'amount_per_100g').fillna(0)

In [164]:
food_vit_pivot = food_vit_pivot.reset_index()

In [194]:
food_vit_pivot['SUM'] = (food_vit_pivot['Vitamin A, RAE'] +
                         food_vit_pivot['Vitamin C, total ascorbic acid']+
                        food_vit_pivot['Vitamin E'] +
                        food_vit_pivot['Vitamin E (alpha-tocopherol)'] +
                        food_vit_pivot['Zinc, Zn'])

In [195]:
food_vit_pivot.sort_values('Vitamin A, RAE', ascending=False)

name,description,"Vitamin A, RAE","Vitamin C, total ascorbic acid",Vitamin E,Vitamin E (alpha-tocopherol),"Zinc, Zn",AVG,SUM
60004,"Fish oil, cod liver",0.030000,0.0000,0.0,0.00000,0.00000,0.006000,0.030000
21702,"Beef, New Zealand, imported, variety meats and...",0.028319,0.0000,0.0,0.00184,0.00300,0.006632,0.033159
166680,"Veal, variety meats and by-products, liver, co...",0.021145,0.0011,0.0,0.00068,0.01123,0.006831,0.034155
21697,"Beef, New Zealand, imported, variety meats and...",0.021014,0.0000,0.0,0.00128,0.00340,0.005139,0.025694
166681,"Veal, variety meats and by-products, liver, co...",0.020074,0.0007,0.0,0.00060,0.01190,0.006655,0.033274
...,...,...,...,...,...,...,...,...
61043,"GARDEIN, CRISPY CHICK'N",0.000000,0.0000,0.0,0.00000,0.00000,0.000000,0.000000
61044,"GARDEIN, CRISPY CHICK'N SLIDERS",0.000000,0.0000,0.0,0.00000,0.00000,0.000000,0.000000
61045,"GARDEIN, CRISPY FINGERS, CHIPOTLE, LIME",0.000000,0.0013,0.0,0.00000,0.00000,0.000260,0.001300
61046,"GARDEIN, CRISPY FINGERS, CHIPOTLE, LIME, CHIPO...",0.000000,0.0013,0.0,0.00000,0.00000,0.000260,0.001300


In [196]:
food_vit_pivot[food_vit_pivot['Vitamin C, total ascorbic acid'] < 100].sort_values('SUM', ascending= False)

name,description,"Vitamin A, RAE","Vitamin C, total ascorbic acid",Vitamin E,Vitamin E (alpha-tocopherol),"Zinc, Zn",AVG,SUM
106464,ORANGE FLAVORED LOW CALORIE VITAMIN & MINERAL ...,0.0,33.0000,0.0,0.0,0.5,6.70000,33.5000
58727,"FRUIT JUICE, APPLE",0.0,30.0000,0.0,0.0,0.0,6.00000,30.0000
154568,"TANGERINE LEMONADE LOW CALORIE DRINK MIX, TANG...",0.0,28.5714,0.0,0.0,0.0,5.71428,28.5714
154569,TANGERINE LEMONADE VITAMIN C LOW CALORIE DRINK...,0.0,28.5714,0.0,0.0,0.0,5.71428,28.5714
106587,"ORANGE PINEAPPLE LOW CALORIE DRINK MIX, ORANGE...",0.0,28.5714,0.0,0.0,0.0,5.71428,28.5714
...,...,...,...,...,...,...,...,...
112105,ORIGINAL REDUCED SODIUM CHICKEN BROTH,0.0,0.0000,0.0,0.0,0.0,0.00000,0.0000
54001,"FESTY WOW!, CHOCOLATE SANDWICH COOKIES",0.0,0.0000,0.0,0.0,0.0,0.00000,0.0000
112103,ORIGINAL RECIPE WHOLE WHEAT ORGANIC PITA BREAD,0.0,0.0000,0.0,0.0,0.0,0.00000,0.0000
112102,"ORIGINAL RECIPE TAPIOCA PUDDING, ORIGINAL RECIPE",0.0,0.0000,0.0,0.0,0.0,0.00000,0.0000


In [197]:
food_vit_pivot.to_csv('Data/skin_break_food.csv')

In [34]:
food[food['name'] == 'Iron, Fe']

,fdc_id,data_type,description,publication_date,id,nutrient_id,amount,data_points,derivation_id,name,unit_name,nutrient_nbr,rank
4,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",2019-04-01,1283678,1089,2.12,1.0,46.0,"Iron, Fe",MG,303.0,5400.0
18,167513,sr_legacy_food,"Pillsbury, Cinnamon Rolls with Icing, refriger...",2019-04-01,1283692,1089,1.93,1.0,47.0,"Iron, Fe",MG,303.0,5400.0
43,167515,sr_legacy_food,"George Weston Bakeries, Thomas English Muffins",2019-04-01,1283718,1089,1.40,1.0,47.0,"Iron, Fe",MG,303.0,5400.0
66,167516,sr_legacy_food,"Waffles, buttermilk, frozen, ready-to-heat",2019-04-01,1283769,1089,6.04,2.0,1.0,"Iron, Fe",MG,303.0,5400.0
177,167517,sr_legacy_food,"Waffle, buttermilk, frozen, ready-to-heat, toa...",2019-04-01,1283957,1089,6.59,6.0,1.0,"Iron, Fe",MG,303.0,5400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7533302,2007121,sub_sample_food,"mushrooms, enoki",2021-10-15,24473355,1089,1.25,1.0,1.0,"Iron, Fe",MG,303.0,5400.0
7533320,2007139,sub_sample_food,"mushrooms, enoki",2021-10-15,24473373,1089,1.43,1.0,1.0,"Iron, Fe",MG,303.0,5400.0
7533338,2007157,sub_sample_food,"mushrooms, enoki",2021-10-15,24473391,1089,1.34,1.0,1.0,"Iron, Fe",MG,303.0,5400.0
7533356,2007175,sub_sample_food,"mushrooms, enoki",2021-10-15,24473409,1089,1.23,1.0,1.0,"Iron, Fe",MG,303.0,5400.0


In [36]:
food_nona = food.dropna()

In [39]:
food_nona = food_nona[(food_nona['unit_name'] == 'G') | (food_nona['unit_name'] == 'MG')]

In [169]:
food_nona = food_nona[(food_nona['name'] == 'Protein') | (food_nona['name'] == 'Iron, Fe')]

In [170]:
food_nona

,fdc_id,data_type,description,publication_date,id,nutrient_id,amount,data_points,derivation_id,name,unit_name,nutrient_nbr,rank,amount_per_100g
0,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",2019-04-01,1283674,1003,5.88,1.0,46.0,Protein,G,203.0,600.0,5.88000
4,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",2019-04-01,1283678,1089,2.12,1.0,46.0,"Iron, Fe",MG,303.0,5400.0,0.00212
14,167513,sr_legacy_food,"Pillsbury, Cinnamon Rolls with Icing, refriger...",2019-04-01,1283688,1003,4.34,1.0,47.0,Protein,G,203.0,600.0,4.34000
18,167513,sr_legacy_food,"Pillsbury, Cinnamon Rolls with Icing, refriger...",2019-04-01,1283692,1089,1.93,1.0,47.0,"Iron, Fe",MG,303.0,5400.0,0.00193
32,167514,sr_legacy_food,"Kraft Foods, Shake N Bake Original Recipe, Coa...",2019-04-01,1283712,1003,6.10,1.0,1.0,Protein,G,203.0,600.0,6.10000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7533302,2007121,sub_sample_food,"mushrooms, enoki",2021-10-15,24473355,1089,1.25,1.0,1.0,"Iron, Fe",MG,303.0,5400.0,0.00125
7533320,2007139,sub_sample_food,"mushrooms, enoki",2021-10-15,24473373,1089,1.43,1.0,1.0,"Iron, Fe",MG,303.0,5400.0,0.00143
7533338,2007157,sub_sample_food,"mushrooms, enoki",2021-10-15,24473391,1089,1.34,1.0,1.0,"Iron, Fe",MG,303.0,5400.0,0.00134
7533356,2007175,sub_sample_food,"mushrooms, enoki",2021-10-15,24473409,1089,1.23,1.0,1.0,"Iron, Fe",MG,303.0,5400.0,0.00123


In [171]:
food_nona['amount_per_100g'] = np.where(food_nona['unit_name'] != 'MG', food_nona['amount'], food_nona['amount']/1000)

<ipython-input-171-b0360b8e4d14>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_nona['amount_per_100g'] = np.where(food_nona['unit_name'] != 'MG', food_nona['amount'], food_nona['amount']/1000)


In [174]:
food_nona_pivot = food_nona.drop(columns = ['fdc_id',
                             'data_type',
                             'publication_date',
                             'id',
                             'nutrient_id',
                             'data_points',
                             'derivation_id',
                             'nutrient_nbr', 
                            'rank'])

In [175]:
food_nona_pivot.head()

,description,amount,name,unit_name,amount_per_100g
0,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",5.88,Protein,G,5.88000
4,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",2.12,"Iron, Fe",MG,0.00212
14,"Pillsbury, Cinnamon Rolls with Icing, refriger...",4.34,Protein,G,4.34000
18,"Pillsbury, Cinnamon Rolls with Icing, refriger...",1.93,"Iron, Fe",MG,0.00193
32,"Kraft Foods, Shake N Bake Original Recipe, Coa...",6.10,Protein,G,6.10000


In [176]:
food_nona_pivot = food_nona_pivot.pivot_table(index = 'description', columns= 'name', values = 'amount_per_100g')

In [181]:
food_nona_pivot = food_nona_pivot.dropna()

In [186]:
food_nona_pivot['Protein and Iron'] = (food_nona_pivot['Iron, Fe'] + food_nona_pivot['Protein'])

<ipython-input-186-819a2cec25d1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_nona_pivot['Protein and Iron'] = (food_nona_pivot['Iron, Fe'] + food_nona_pivot['Protein'])


In [189]:
food_nona_pivot = food_nona_pivot.sort_values('Protein and Iron',ascending = False).reset_index()

In [17]:
food_nona.to_csv('Data/food_data.csv')

In [27]:
food_nona = food_nona.sort_values('publication_date', ascending=False).drop_duplicates('description')

In [40]:
food_nona.to_csv('Data/food_data_update.csv')

OSError: [Errno 22] Invalid argument: 'Data/food_data_update.csv'

In [191]:
food_nona_pivot.to_csv('Data/fall_foods.csv')